<a href="https://colab.research.google.com/github/KJ22222/ESSA/blob/main/10_3_%EA%B3%BC%EC%A0%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#라이브러리

In [3]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "unsupervised_learning"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

#9장 비지도 학습
- 군집화
- k-평균
- DBSCAN
- 가우시안 혼합

#9.1 군집
- 비슷한 샘플을 구별해 하나의 클러스터 또는 비슷한 샘플의 그룹으로 할당하는 작업
- 보편적인 알고리즘은 없음
- 레이블이 없는데 만드는 것임

###9.1.1 k-평균
- 반복 몇 번으로 이런 종류의 데이터셋을 빠르고 효율적으로 클러스터로 묶을 수 있는 간단한 알고리즘
- 각 클러스터의 중심(센트로이드)을 찾고 가장 가까운 클러스터에 샘플을 할당

In [4]:

blob_centers = np.array(
    [[ 0.2,  2.3],
     [-1.5 ,  2.3],
     [-2.8,  1.8],
     [-2.8,  2.8],
     [-2.8,  1.3]])

blob_std = np.array([0.4, 0.3, 0.1, 0.1, 0.1])

In [5]:
from sklearn.datasets import make_blobs
X, _ = make_blobs(n_samples=2000, centers=blob_centers,
                  cluster_std=blob_std, random_state=7)

In [6]:
from sklearn.cluster import KMeans

In [7]:
k = 5
kmeans = KMeans(n_clusters=k, random_state=42)
kmeans.fit(X)

KMeans(n_clusters=5, random_state=42)

알고리즘이 찾을 클러스터의 개수 K개를 지정해야함

In [8]:
kmeans.labels_

array([4, 0, 1, ..., 2, 1, 0], dtype=int32)

In [9]:
kmeans.cluster_centers_

array([[-2.80389616,  1.80117999],
       [ 0.20876306,  2.25551336],
       [-2.79290307,  2.79641063],
       [-1.46679593,  2.28585348],
       [-2.80037642,  1.30082566]])

In [10]:
X_new = np.array([[0, 2], [3, 2], [-3, 3], [-3, 2.5]])

kmeans.predict(X_new)

array([1, 1, 2, 2], dtype=int32)

####소프트 군집
- 샘플과 각 군집 사이의 관계를 점수로 부여

In [11]:
kmeans.transform(X_new)

array([[2.81093633, 0.32995317, 2.9042344 , 1.49439034, 2.88633901],
       [5.80730058, 2.80290755, 5.84739223, 4.4759332 , 5.84236351],
       [1.21475352, 3.29399768, 0.29040966, 1.69136631, 1.71086031],
       [0.72581411, 3.21806371, 0.36159148, 1.54808703, 1.21567622]])

####k-평균 알고리즘

- 모든 샘플에 대해 가장 가까운 센트로이드 지정
- 각각의 센트로이드와 연결된 샘플들의 평균으로 해당 센트로이도 위치 수

센트로이드 초기화 방법

In [12]:
good_init = np.array([[-3, 3], [-3, 2], [-3, 1], [-1, 2], [0, 2]])

kmeans = KMeans(n_clusters=5, init=good_init, n_init=1, random_state=42)
kmeans.fit(X)
kmeans.inertia_

211.5985372581684

- n_init 매개변수로 랜던 초기화 횟수를 조절함
- 초기화 횟수 중에 최전힁 솔류선을 반환함
- 이는 이너셔 라는 성능지표가 가장 낮은걸 사용

In [13]:
kmeans.inertia_

211.5985372581684

In [14]:

kmeans.score(X)

-211.59853725816836

score() 메서드는 관성의 음숫값을 반환한다. 이유는 "더 높은 점수가 더 좋다"의 원칙을 따라야 하기 때문이다.

###k평균 속도 개선과 미니배치 k-평균
- 전체 데이터셋을 사용해 반복하지 않고 이 알고리즘은 각 반복마다 미니배치를 사용해 센트로이드를 조금씩 이동 시킴

In [15]:
from sklearn.cluster import MiniBatchKMeans

####최적의 클러스터 개수 찾기
- 실루엣 점수로 구함
- 이는 실루엣 계수의 평균

In [16]:
from sklearn.metrics import silhouette_score

In [17]:
silhouette_score(X, kmeans.labels_)

0.655517642572828

###9.1.3 군집을 이용한 이미지 분할
- 이미지를 세그먼트 여러 개로 분할하는 작업
- 시맨틱 분할에서는 동일한 종류의 물체에 속한 모든 픽셀은 같은 세그먼트에 할당

색상분할

###9.1.4 군집을 사용한 전처리
- 차원 축소에 효과적인 방법

In [18]:
from sklearn.datasets import load_digits

X_digits, y_digits = load_digits(return_X_y=True)

In [19]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_digits, y_digits, random_state=42)

In [20]:
from sklearn.linear_model import LogisticRegression

log_reg = LogisticRegression(multi_class="ovr", solver="lbfgs", max_iter=5000, random_state=42)
log_reg.fit(X_train, y_train)

LogisticRegression(max_iter=5000, multi_class='ovr', random_state=42)

In [21]:

log_reg.score(X_test, y_test)

0.9688888888888889

k-군집으로 전처리 해서 돌려보기

In [22]:
from sklearn.pipeline import Pipeline

In [23]:

pipeline = Pipeline([
    ("kmeans", KMeans(n_clusters=50, random_state=42)),
    ("log_reg", LogisticRegression(multi_class="ovr", solver="lbfgs", max_iter=5000, random_state=42)),
])
pipeline.fit(X_train, y_train)

Pipeline(steps=[('kmeans', KMeans(n_clusters=50, random_state=42)),
                ('log_reg',
                 LogisticRegression(max_iter=5000, multi_class='ovr',
                                    random_state=42))])

In [24]:

pipeline.score(X_test, y_test)

0.9777777777777777

최적의 클러스터 개수 k 찾기

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = dict(kmeans__n_clusters=range(2, 100))
grid_clf = GridSearchCV(pipeline, param_grid, cv=3, verbose=2)
grid_clf.fit(X_train, y_train)

Fitting 3 folds for each of 98 candidates, totalling 294 fits
[CV] END ...............................kmeans__n_clusters=2; total time=   1.2s
[CV] END ...............................kmeans__n_clusters=2; total time=   1.0s
[CV] END ...............................kmeans__n_clusters=2; total time=   0.5s
[CV] END ...............................kmeans__n_clusters=3; total time=   0.3s
[CV] END ...............................kmeans__n_clusters=3; total time=   0.5s
[CV] END ...............................kmeans__n_clusters=3; total time=   0.4s
[CV] END ...............................kmeans__n_clusters=4; total time=   0.4s
[CV] END ...............................kmeans__n_clusters=4; total time=   0.4s
[CV] END ...............................kmeans__n_clusters=4; total time=   0.5s
[CV] END ...............................kmeans__n_clusters=5; total time=   0.5s
[CV] END ...............................kmeans__n_clusters=5; total time=   0.4s
[CV] END ...............................kmeans_

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


[CV] END ..............................kmeans__n_clusters=55; total time=   9.7s
[CV] END ..............................kmeans__n_clusters=56; total time=   6.9s
[CV] END ..............................kmeans__n_clusters=56; total time=   9.1s
[CV] END ..............................kmeans__n_clusters=56; total time=  12.8s
[CV] END ..............................kmeans__n_clusters=57; total time=  11.8s
[CV] END ..............................kmeans__n_clusters=57; total time=   8.2s
[CV] END ..............................kmeans__n_clusters=57; total time=   8.1s
[CV] END ..............................kmeans__n_clusters=58; total time=   9.9s
[CV] END ..............................kmeans__n_clusters=58; total time=   8.3s
[CV] END ..............................kmeans__n_clusters=58; total time=   6.5s
[CV] END ..............................kmeans__n_clusters=59; total time=   7.1s
[CV] END ..............................kmeans__n_clusters=59; total time=   7.4s
[CV] END ...................

In [ ]:
grid_clf.best_params_

In [ ]:
grid_clf.score(X_test, y_test)

###9.1.5  군집화 활용: 준지도학습

In [ ]:
n_labeled = 50

In [ ]:
log_reg = LogisticRegression(multi_class="ovr", solver="lbfgs", random_state=42)
log_reg.fit(X_train[:n_labeled], y_train[:n_labeled])

log_reg.score(X_test, y_test)

훈련 세트를 50개의 클러스터로 모르고 각 클러스터에서 센트로이드에 가장 가까운 이미지를 찾음->대표 이미지

In [ ]:
k = 50

kmeans = KMeans(n_clusters=k, random_state=42)
X_digits_dist = kmeans.fit_transform(X_train)

In [36]:
representative_digit_idx = np.argmin(X_digits_dist, axis=0)  # 50개의 대표이미지 인덱스 확인
X_representative_digits = X_train[representative_digit_idx]  # 50개의 대표이미지 지정

In [37]:
y_representative_digits = y_train[representative_digit_idx]

In [ ]:
y_train[representative_digit_idx]

레이블 전파 1

In [39]:
y_train_propagated = np.empty(len(X_train), dtype=np.int32)

for i in range(k):
    y_train_propagated[kmeans.labels_==i] = y_representative_digits[i]

In [40]:
log_reg = LogisticRegression(multi_class="ovr", solver="lbfgs", max_iter=5000, random_state=42)
log_reg.fit(X_train, y_train_propagated)

log_reg.score(X_test, y_test)

0.9333333333333333

레이블 전파 2

In [41]:
X_cluster_dist = X_digits_dist[np.arange(len(X_train)), kmeans.labels_]

In [42]:
percentile_closest = 20

for i in range(k):
    in_cluster = (kmeans.labels_ == i)                 # 군집별 샘플 대상
    cluster_dist = X_cluster_dist[in_cluster]
    cutoff_distance = np.percentile(cluster_dist, percentile_closest)   # 군집별 센트로이드 근접도 상위 20% 경곗값
    above_cutoff = (X_cluster_dist > cutoff_distance)  # 군집별 센트로이드 근접도 상위 20% 이내 샘플 대상
    X_cluster_dist[in_cluster & above_cutoff] = -1

### 9.1.6 DBSCAN

In [43]:
from sklearn.cluster import DBSCAN

In [44]:
dbscan = DBSCAN(eps=0.05, min_samples=5)
dbscan.fit(X)

DBSCAN(eps=0.05)

In [45]:
np.unique(dbscan.labels_)

array([-1,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15,
       16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28])

In [46]:
dbscan.labels_[:10]

array([ 0,  0, -1,  0,  0,  0,  0,  0,  1,  0])

In [47]:
len(dbscan.core_sample_indices_)

1296

##9.2 가우시안 혼합
- 샘플이 파라미터가 알려지지 않은 여러 개의 혼합된 가우시안 분포에서 생성되었다고 가정하는 확률 모델

In [48]:
from sklearn.mixture import GaussianMixture

In [49]:
gm = GaussianMixture(n_components=3, n_init=10, random_state=42)
gm.fit(X)

GaussianMixture(n_components=3, n_init=10, random_state=42)

In [50]:
gm.weights_

array([0.19978216, 0.59989596, 0.20032189])

In [51]:
gm.means_

array([[-1.48543693,  2.28573643],
       [-2.80278963,  1.96665812],
       [ 0.19937187,  2.25813691]])

In [52]:
gm.covariances_

array([[[ 8.79984367e-02,  1.46541758e-04],
        [ 1.46541758e-04,  9.01630979e-02]],

       [[ 9.95376970e-03,  7.16529541e-04],
        [ 7.16529541e-04,  3.96947649e-01]],

       [[ 1.46166449e-01, -4.84946828e-03],
        [-4.84946828e-03,  1.56280672e-01]]])

In [53]:
gm.converged_

True

In [54]:

gm.n_iter_

3

In [55]:
gm.predict(X)
gm.predict_proba(X)#소프트 군집

array([[1.19140197e-006, 9.99998809e-001, 5.11211601e-015],
       [1.56262666e-006, 9.99998437e-001, 6.43557803e-016],
       [7.51207529e-016, 4.58103106e-307, 1.00000000e+000],
       ...,
       [8.44387449e-006, 9.99991556e-001, 7.53349575e-015],
       [7.54961300e-008, 6.39866293e-202, 9.99999925e-001],
       [2.66779387e-004, 9.99733221e-001, 3.59569371e-013]])

확률 밀도 함수

In [56]:
gm.score_samples(X)

array([-0.62897413,  0.13004267, -7.15363365, ..., -0.3150245 ,
       -1.62809493, -0.94509528])

In [57]:

np.exp(gm.score_samples(X_new))

array([1.47181457e-01, 4.30497825e-13, 5.40438978e-02, 1.47601946e-01])

In [58]:
resolution = 100
grid = np.arange(-10, 10, 1 / resolution)
xx, yy = np.meshgrid(grid, grid)

X_full = np.vstack([xx.ravel(), yy.ravel()]).T

In [59]:
# exp() 함수를 적용하여 log() 함수를 상쇄시킴
pdf = np.exp(gm.score_samples(X_full))

# 격자의 크기를 확률밀도와 곱하기
pdf_probas = pdf * (1 / resolution) ** 2
pdf_probas.sum()

1.0000000000000402

결정경계와 밀도 등고선
- plot_gaussian_mixture() 함수는 가우시안 혼합 모델이 알아낸 군집 결정경계와 샘플의 로그밀도 등고선(density contours)을 그린다.

In [60]:
from matplotlib.colors import LogNorm

def plot_gaussian_mixture(clusterer, X, resolution=1000, show_ylabels=True):
    # 로그밀도 등고선 그리기
    mins = X.min(axis=0) - 0.1
    maxs = X.max(axis=0) + 0.1
    xx, yy = np.meshgrid(np.linspace(mins[0], maxs[0], resolution),
                         np.linspace(mins[1], maxs[1], resolution))
    # score_samples가 기본적으로 음수이기에 양수로 변환함
    Z = -clusterer.score_samples(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)

    plt.contourf(xx, yy, Z,
                 norm=LogNorm(vmin=1.0, vmax=30.0),
                 levels=np.logspace(0, 2, 12))
    plt.contour(xx, yy, Z,
                norm=LogNorm(vmin=1.0, vmax=30.0),
                levels=np.logspace(0, 2, 12),
                linewidths=1, colors='k')

    # 결정경계 그리기: 빨강 파선
    Z = clusterer.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    plt.contour(xx, yy, Z, linewidths=2, colors='r', linestyles='dashed')
    
    # 데이터 산점도 및 센트로이드 그리기
    plt.plot(X[:, 0], X[:, 1], 'k.', markersize=2)
    plot_centroids(clusterer.means_, clusterer.weights_)

    plt.xlabel("$x_1$", fontsize=14)
    if show_ylabels:
        plt.ylabel("$x_2$", fontsize=14, rotation=0)
    else:
        plt.tick_params(labelleft=False)

###9.2.4 이상치 탐지와 특이치 탐지를 위한 다른 알고리즘

- pca:간단하고 종종 매우 효과적인 이상치 탐지 기법
- fast-mcd:데이터셋을 정제할 때 사용됨.보통 샘플이 하나의 가우시안 분포에서 생성되었다고 가정함
- 아이솔레이션 포레스트: 고차원 데이터셋에서 이상치 감지를 위한 효율적인 알고리즘,무작위로 성장한 결정 트리로 구성된 랜덤 포레스트를 만듭니다.
- lof: 주어진 샘플 주위의 밀도와 이웃 주위의 밀도를 비교함.이상치는 종종 k개의 최근접 이웃보다 더 격리됨
- one-class svm:원본 공간에서는 모든 샘플을 둘러싼 작은 영역을 찾음